# IsoNet
------
This will be used to create the model for predicting precipitable stable water isotopes. Only creates and exports the model and trains it. Is not used to predict.

Table of Contents:
1. [Importing Data](#importing-data)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Creation](#model-creation)
4. [Model Training](#model-training)
5. [Model Export](#model-export)

In [131]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime
import matplotlib.pyplot as plt

# Tensorflow and keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

## Importing Data
Importing the data, and then splitting it into features and labels

In [132]:
# Load in the data, separate the features and labels
data = pd.read_csv(r'Isoscape_Data.csv')
features = data.drop(['H2avg', 'dex', 'O18Avg', 'Station'], axis=1)
labels = data['O18Avg']

# Adjusting the date time to be int values where Janurary 1st, of year 1 is 1
features['Date'] = pd.to_datetime(features['Date'])
features['Date'] = features['Date'].map(datetime.datetime.toordinal)

numFeatures = features.shape[1]

## Data Preprocessing
Preprocessing the data, including:
* Convert into numpy arrays
* Split into training and testing sets
* Convert into tensorflow Datasets
* Windowing the data (Skipping for now, will do later) 
* Standardizing the data

In [133]:
# Split the data into training and testing sets
splitIndex = int(0.8 * len(features))
xTrain = features[:splitIndex]
xTest = features[splitIndex:]
yTrain = labels[:splitIndex]
yTest = labels[splitIndex:]

# Scale the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

# Convert the data into numpy arrays
xTrain = np.array(xTrain)
xTest = np.array(xTest)
yTrain = np.array(yTrain)
yTest = np.array(yTest)

# Convert traindata into tensor dataset
trainData = tf.data.Dataset.from_tensor_slices((xTrain, yTrain))

## Model Creation
Creating the model, including:
* Creating the model architecture
* Compiling the model

Model architecture:
* Input layer
* 1 LSTM layer
* 2 Dense layers
* Output layer

In [134]:
numNeuorns = 64
model = Sequential()
model.add(InputLayer(input_shape=(numFeatures, 1)))
model.add(LSTM(numNeuorns))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

## Model Training
Training and evaluating the model

In [135]:
es = EarlyStopping(monitor='mean_absolute_error', patience=10)

batchSize = 64
model.fit(
    trainData.batch(batchSize),
    batch_size=batchSize,
    epochs=1000,
    callbacks=[es]
)

Epoch 1/1000


23/23 [==============================] - 3s 7ms/step - loss: 386.0189 - root_mean_squared_error: 19.6474 - mean_absolute_error: 17.7332
Epoch 2/1000
23/23 [==============================] - 0s 7ms/step - loss: 338.6252 - root_mean_squared_error: 18.4018 - mean_absolute_error: 16.2355
Epoch 3/1000
23/23 [==============================] - 0s 8ms/step - loss: 112.0202 - root_mean_squared_error: 10.5840 - mean_absolute_error: 8.0789
Epoch 4/1000
23/23 [==============================] - 0s 8ms/step - loss: 49.1132 - root_mean_squared_error: 7.0081 - mean_absolute_error: 5.4688
Epoch 5/1000
23/23 [==============================] - 0s 7ms/step - loss: 37.9007 - root_mean_squared_error: 6.1564 - mean_absolute_error: 4.9286
Epoch 6/1000
23/23 [==============================] - 0s 7ms/step - loss: 33.6112 - root_mean_squared_error: 5.7975 - mean_absolute_error: 4.6627
Epoch 7/1000
23/23 [==============================] - 0s 7ms/step - loss: 32.3590 - root_mean_squared_error: 5.6885 - mean_absolu

## Model Export

In [137]:
# Printing how well the model did on the test data
print(model.evaluate(xTest, yTest))

12/12 [==============================] - 0s 3ms/step - loss: 14.2762 - root_mean_squared_error: 3.7784 - mean_absolute_error: 2.8717
[14.276230812072754, 3.7783899307250977, 2.871666431427002]


In [16]:
# Create new pd DataFrame with the predictions and actual values including the features
predictions = model.predict(xTest)
df = features.loc[splitIndex:]
df['Predictions'] = predictions
df['Actual'] = yTest

# Export the data to a csv file
df.to_csv('results_test.csv', index=False)

12/12 [==============================] - 0s 3ms/step


/tmp/ipykernel_661/3509207634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predictions'] = predictions
/tmp/ipykernel_661/3509207634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual'] = yTest


In [17]:
# Save the model
model.save('printModel.keras')

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                16896     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25281 (98.75 KB)
Trainable params: 25281 (98.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
